In [1]:
import pydrake.solvers.mathematicalprogram as mp
import scipy.interpolate as interp
import numpy as np

In [11]:
import scipy

In [19]:
c = np.array([[[3,4], [3,4]], [[2, 3], [2, 3]], [[1, 2], [1, 2]]])
x = np.array([0, 1, 2])

In [20]:
c.shape

(3, 2, 2)

In [21]:
p = interp.PPoly(c, x)

In [26]:
p(1.1)

array([ 1.23,  2.34])

In [27]:
p(0.1)

array([ 1.23,  2.34])

In [28]:
prog = mp.MathematicalProgram()
C = prog.NewContinuousVariables(3, 4, "x")

In [29]:
C.shape

(3, 4)

In [30]:
def piecewise_polynomial_variable(prog, domain, dimension, degree):
    C = [prog.NewContinuousVariables(degree + 1, len(domain) - 1) for _ in range(dimension)]
    C = np.stack(C, axis=2)
    assert C.shape == (degree + 1, len(domain) - 1, dimension)
    return interp.PPoly(C, domain)

In [31]:
np.linspace?

In [33]:
prog = mp.MathematicalProgram()
position = piecewise_polynomial_variable(prog, np.linspace(0, 1, 10), 1, 2)

TypeError: float() argument must be a string or a number